# Geographic Heatmap in Python with Folium

__Python 3.4+__  

This notebook has a [companion blog post](https://alcidanalytics.com/p/geographic-heatmap-in-python). In summary it's an example of using folium to generate interactive geographic heatmaps. It also shows how to add arbitrary shapefiles and geojson to leaflet maps with geopandas.

<a href="https://alcidanalytics.com/p/geographic-heatmap-in-python"><img src="images/geo_heatmap_banner.png"></a>

__The tools__: 
- [folium](https://github.com/python-visualization/folium) a Python library that outputs HTML with leaflet.js maps
- [geopandas](geopandas.org) extends the datatypes used by pandas to allow spatial operations on geometric types

__The data__: 
- shapefiles and congressional boundaries from [census.gov](https://www.census.gov/geo/maps-data/)
- campaign contribution data from [followthemoney.org](http://www.followthemoney.org/our-data/about-our-data/)

## Load Campaign Contribution Data

These data were aggregated from an export from followthemoney.org. How, will be described in a separate notebook and blog post.

In [1]:
import pandas as pd 

for_map = pd.read_csv('data/campaign_contributions_for_map.tsv', sep='\t')

for_map.sort_values('Amount', ascending=False).head()

,City,State,lat,lon,Amount
509,WASHINGTON,DC,38.907192,-77.036871,1918808.40
412,ROCHESTER,NY,43.160000,-77.610000,195067.39
6,ALEXANDRIA,VA,38.804835,-77.046921,185438.32
25,ARLINGTON,VA,38.879970,-77.106770,138900.00
346,NEW YORK,NY,40.710000,-73.990000,130620.00


In [2]:
for_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549 entries, 0 to 548
Data columns (total 5 columns):
City      549 non-null object
State     549 non-null object
lat       549 non-null float64
lon       549 non-null float64
Amount    549 non-null float64
dtypes: float64(3), object(2)
memory usage: 21.5+ KB


## Load 23rd Congr. District Shapefile

In [3]:
import geopandas as gpd
import folium

congr_districts = gpd.read_file('zip://'+'data/cb_2015_us_cd114_20m.zip')

# Set datum and projection info for census.gov 2015 Tiger data
congr_districts.crs = {'datum': 'NAD83', 'ellps': 'GRS80', 'proj':'longlat', 'no_defs':True}

# Filter out all but the district of interest
district23 = congr_districts[ congr_districts.GEOID == '3623' ]  # 36 = NY, 23 = District

Initialize a folium map and see what the default crs data is in case we need to convert the shapefile we loaded

In [4]:
distric_map = folium.Map(location=[42.5, -76], zoom_start=7, tiles='cartodbpositron' )
print('default map crs: ',distric_map.crs)

default map crs:  EPSG3857


In [5]:
# convert it to the projection of our folium openstreetmap
district23 = district23.to_crs({'init':'epsg:3857'})  

Try plotting the congressional district boundary.

(Leaflet.js maps don't render in github notebook previews, [check out the blog post](https://alcidanalytics.com/p/geographic-heatmap-in-python) if you want to see the live interactive maps without having to run this notebook)

In [6]:
folium.GeoJson(district23).add_to(distric_map)
distric_map

## Heatmap

In [8]:
import folium
from folium.plugins import HeatMap

max_amount = float(for_map['Amount'].max())

hmap = folium.Map(location=[41.0, -75.5], zoom_start=7, )

hm_wide = HeatMap( list(zip(for_map.lat.values, for_map.lon.values, for_map.Amount.values)), 
                   min_opacity=0.2,
                   max_val=max_amount,
                   radius=17, blur=15, 
                   max_zoom=1, 
                 )

folium.GeoJson(district23).add_to(hmap)
hmap.add_child(hm_wide)

In [9]:
hmap.save(os.path.join('results', 'geo_heatmap.html'))

# Saving png's of Folium Maps

https://github.com/python-visualization/folium/issues/35

These aren't tested, just pasted here as a starting point should someone need them.

### Using cutycapt

In [ ]:
import os
import subprocess
outdir = "results" # this directory has to exist..
#hmap.save("tmp.html")  # heatmap saved above
url = "file://{}/results/geo_heatmap.html".format(os.getcwd())
outfn = os.path.join(outdir,"geo_heatmap.png")
subprocess.check_call(["cutycapt","--url={}".format(url), "--out={}".format(outfn)])

### Using selenium

In [ ]:
import os
import time
from selenium import webdriver

fn = os.path.join('results', 'geo_heatmap.html')
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
#hmap.save("tmp.html")  # heatmap saved above

browser = webdriver.Firefox()
browser.get(tmpurl)
time.sleep(5)  #Give the map tiles some time to load
browser.save_screenshot('geo_heatmap.png')
browser.quit()